# On the viability of crowdsourcing NLP annotations in healthcare

In [1]:
__author__  = 'Bruno Godefroy'

__TL;DR__ For most annotation tasks in healthcare, we must rely on trained experts, who are always in high-demand, creating a bottleneck for system development. Can we address this limitation by crowdsourcing the easier annotation tasks in this space? We report on an experiment addressing this question, with a focus on how to identify problematic workers and infer reliable annotations from noisy crowd work.

## Overview

Obtaining high-quality annotations is a bottleneck for all natural language processing applications. The challenges are especially severe in healthcare, where we rely on annotators who have expertise in the practice of medicine and in understanding medical texts, and who are authorized to access sensitive data. Such annotators are in high-demand and thus demand high prices. The question then arises whether we can ease the burden on these annotators by crowdsourcing the less demanding annotation tasks involving publicly-available data, reserving the experts for where they are truly needed.

This post reports on a crowdsourcing experiment we ran to explore this issue. We defined a reasonably nuanced span-identification task, and launched it on [Figure Eight](https://www.figure-eight.com). As expected, the output was noisy, as a result of the highly variable pool of annotators we tapped into. To infer quality labels from this noisy process, we used a straightforward application of Expectation-Maximization, with quite good results, suggesting that crowdsourcing is an effective tool for obtaining annotations for at least some NLP problems in healthcare.

## Crowdsourcing task definition

The publicly available [FDA Drug Labels data set](https://labels.fda.gov) is a rich source of information about how drugs relate to disease states (among other things). Since it's a public dataset, we don't have to address privacy concerns, which are of course another limiting factor when it comes to annotating data in healthcare.

For our pilot task, we decided to focus on developing annotations to facilitate automatic extraction of the core drug–disease relationships expressed in these labels, as exemplified in <a href='#fig1'>figure 1</a>. This is [a problem we've worked on before](https://roamanalytics.com/2018/02/02/retrofitting-distributional-embeddings-to-knowledge-graphs-with-functional-relations/), so we have a good sense for what information should be extracted. 

<div id='fig1'>
    <img src="figures/annotation_example.png" style='width: 600px; margin-top: 20px;'/>
    <p style='text-align: center; font-style: italic; font-size: 13px; max-width: 400px; margin: 15px auto auto auto;'>Figure 1: Annotated sentences from a drug label.</p>
</div>

We need our NLP model to identify the disease mentions in these texts and determine the sense for those mentions. Our own high-level exploration of the data led us to the label set _prevents_, _treats_, and _is contraindicated for_ as the right set of high-level categories for the mentions. This annotation design is motivated by recent experiments which gave encouraging results on similar tasks [<a href='#ref_maclean'>1</a>, <a href='#ref_tenuto'>2</a>].

For crowdsourcing, we break this down into two separate tasks: identifying disease mentions spans, and assigning categories to identified spans. (Breaking the task down in this way isn't strictly speaking necessary, but it eases the burden on individual annotators). Since the span identification task is more open-ended and challenging, it's the one we explore in this post.

We developed a short annotation manual with examples, had an in-house expert label 50 sentences for quality control, and used the Figure Eight platform to design a simple annotation interface (<a href='#fig2'>figure 2</a>).

<div id='fig2'>
    <img src="figures/screenshot_task1.png" style='width: 610px; margin-top: 20px;'/>
    <p style='text-align: center; font-style: italic; font-size: 13px; max-width: 600px; margin: 5px auto auto auto;'>Figure 2: The interface for our first task. Crowdworkers are asked to select disease, symptoms and injury mentions in a sentence. Some drug and disease mentions are automatically underlined using fixed lexicons to help the workers understand the texts.</p>
</div>

## Who completed our task on Figure Eight?

We launched our task on Figure Eight with 10,000 sentences. It was completed within a few days. The job was done by 451 people from 45 countries, the majority from India and Venezuela. No special qualifications were imposed.

Most workers made just a few contributions in a short period of time, as figures <a href='#fig3'>3</a> and <a href='#fig4'>4</a> show. Half of the sessions (continuous periods of work) lasted less than 20 minutes; at the upper end, 7% were more than an hour and a half. This is expected given recent studies of crowdworkers' behaviors [<a href='#ref_zhai'>3</a>, <a href='#ref_amerYahia'>4</a>].

<div style='text-align: center;'>
    <div id='fig3' style='margin-right: 25px; display: inline-block; vertical-align: top;'>
        <img src="figures/n_work_sessions.png" style='height: 250px; margin-top: 0px;'/>
        <p style='text-align: center; font-style: italic; margin-top: 5px; font-size: 13px; line-height: 19px;'>Figure 3: Number of work sessions per contributor.</p>
    </div>
    <div id='fig4' style='display: inline-block; vertical-align: top;'>
        <img src="figures/session_duration.png" style='height: 250px; margin-top: 0px;'/>
        <p style='text-align: center; font-style: italic; margin-top: 5px; font-size: 13px; line-height: 19px;'>Figure 4: Work session durations.</p>
    </div>
</div>

## Assessment against gold labels

Given our large and diverse pool of workers, we expect some of them to be unreliable, perhaps due to a lack of expertise or a lack of attention. With Figure Eight, we can supply our own labeled examples for a subset of cases, to help identify and filter out unreliable workers. <a href='#fig5'>Figure 5</a> summarizes the work of 100 annotators who were rejected from our task based on their performance on this gold data.

<div id='fig5'>
    <img src="figures/failure_reasons.png" style='width: 420px; margin-top: 20px;'/>
    <p style='text-align: center; font-style: italic; font-size: 13px; max-width: 480px; margin: 5px auto auto auto;'>Figure 5: Main reason for failing out of our gold-label assessment. Results come from the manual study of the output from 100 workers who didn't pass the test.</p>
</div>

It seems that several workers attempted cheating strategies, such as selecting no text segments at all. Our gold labels help us identify such work. We also see evidence that some workers used auxiliary tools to translate the full Figure Eight interface into another language, including the sentences to annotate, with the result that the responses they submitted were in that language!

## From noisy judgments to crowd truth

Should we blindly trust all the judgments of all the workers who passed our gold-label assessment? Probably not. Some errors are inevitable even for careful workers, and some malicious workers are likely to slip past our assessment against our gold examples. Furthermore, there are bound to be cases that are ambiguous or open to interpretation, leading to multiple right answers that we ourselves might not have fully appreciated, as in [figure 6](#fig6).

<div id='fig6'>
    <img src="figures/ambiguous_sentence.png" style='width: 530px; margin-top: 0px;'/>
    <p style='text-align: center; font-style: italic; font-size: 13px; max-width: 500px; margin: 5px auto auto auto;'>Figure 6: An ambiguous case. Should "feelings of sadness related to winter" be selected as a disease? Or a symptom?</p>
</div>

The most basic step we can take to address these concerns is to collect multiple judgments from many different annotators, in the hope that a consensus emerges. We could define consensus in various ways – e.g., the majority label, or at least 70% agreement on a label (leaving some cases without a label at all). However, these approaches are clearly suboptimal because they make the implicit assumption that all workers are equally trustworthy, by giving equal weight to all of them. To address this, we use a simple application of the [Expectation-Maximization (EM) method](https://en.wikipedia.org/wiki/Expectation–maximization_algorithm) to our matrix of worker judgments.

Our guiding idea is that "*disagreement is not noise but signal*" [[5](#ref_aroyo)]. Assuming the majority is usually right, it's probably worth giving more credit to workers who often agree with each other. Following this intuition, Dawid and Skene [[6](#ref_dawidSkene)] propose a simple model which uses EM to estimate response confidence and workers' reliability in an efficient way. [Figure 7](#fig7) provides an overview of this algorithm, which we state in full in an [appendix](#appendix).

<div id='fig7'>
    <img src="figures/em.png" style='height: 300px; margin-top: 0px;'/>
    <p style='text-align: center; font-style: italic; font-size: 13px; max-width: 400px; margin: 5px auto auto auto;'>Figure 7: The Expectation-Maximization algorithm for inferring labels from crowd work.</p>
</div>

Let's consider a small example where we want to find the responses to five boolean questions using a crowd of four workers. For each question, we collect three True/False (T/F) judgments.

<div id='tab1'>
    <table>
      <tr>
        <th></th>
        <th>Worker 1</th>
        <th>Worker 2 </th>
        <th>Worker 3</th>
        <th>Worker 4</th>
        <th style='background-color: #d6e5d6;'>True response</th>
      </tr>
      <tr>
        <td><b>Question 1</b></td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center;'></td>
        <td style='text-align: center; background-color: #ecf5ec;'>T</td>
      </tr>
      <tr>
        <td><b>Question 2</b></td>
        <td style='text-align: center;'></td>
        <td style='text-align: center;'>F</td>
        <td style='text-align: center;'>F</td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center; background-color: #ecf5ec;'>F</td>
      </tr>
      <tr>
        <td><b>Question 3</b></td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center;'>F</td>
        <td style='text-align: center;'></td>
        <td style='text-align: center; background-color: #ecf5ec;'>T</td>
      </tr>
      <tr>
        <td><b>Question 4</b></td>
        <td style='text-align: center;'>F</td>
        <td style='text-align: center;'></td>
        <td style='text-align: center;'>F</td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center; background-color: #ecf5ec;'>F</td>
      </tr>
      <tr>
        <td><b>Question 5</b></td>
        <td style='text-align: center;'>F</td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center;'></td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center; background-color: #ecf5ec;'>F</td>
      </tr>
    </table>
    
   <p style='text-align: center; font-style: italic; font-size: 13px; max-width: 450px; margin: 10px auto auto auto;'>Table 1: Dummy collected judgments and true response (that we don't have), for each question.</p>
</div>

Worker 1 looks highly reliable: where it contributed annotations, it always agrees with our true response. Workers 2 and 4 have low *precision* in the sense that they sometimes say T when the correct label is F. Conversely, Worker 3 has low *recall* in the sense that he sometimes says F when the correct label is T.

Of course, we crowdsource precisely because we don't have these true responses – we have to try to uncover them given only the matrix of worker judgments. With EM, we do this by jointly estimating the reliability of each worker and the response that maximizes the likelihood of the observed judgments.

[Table 2](#tab2) gives the output of the algorithm for this simple example. The *crowd response* is the one derived from the EM algorithm; T when the maximum likelihood estimate is larger than 0.5, F otherwise. Our implementation ([available on Github](https://github.com/roaminsight/analysis_notebooks/blob/crowdsourcing-blogpost/Blog_posts/develop/2018-07-02-crowdsourcing_annotation/EM_example.py)) converges after 8 iterations. 

<div id='tab2'>
    <table>
      <tr>
        <th></th>
        <th>Maximum likelihood estimate</th>
        <th>Derived crowd response</th>
        <th style='background-color: #d6e5d6;'>True response</th>
      </tr>
      <tr>
        <td><b>Question 1</b></td>
        <td style='text-align: center;'>0.99</td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center; background-color: #ecf5ec;'>T</td>
      </tr>
      <tr>
        <td><b>Question 2</b></td>
        <td style='text-align: center;'>0.01</td>
        <td style='text-align: center;'>F</td>
        <td style='text-align: center; background-color: #ecf5ec;'>F</td>
      </tr>
      <tr>
        <td><b>Question 3</b></td>
        <td style='text-align: center;'>0.98</td>
        <td style='text-align: center;'>T</td>
        <td style='text-align: center; background-color: #ecf5ec;'>T</td>
      </tr>
      <tr>
        <td><b>Question 4</b></td>
        <td style='text-align: center;'>0.08</td>
        <td style='text-align: center;'>F</td>
        <td style='text-align: center; background-color: #ecf5ec;'>F</td>
      </tr> 
      <tr>
        <td><b>Question 5</b></td>
        <td style='text-align: center;'>0.28</td>
        <td style='text-align: center;'>F</td>
        <td style='text-align: center; background-color: #ecf5ec;'>F</td>
      </tr>
    </table>

<p style='text-align: center; font-style: italic; font-size: 13px; max-width: 450px; margin: 10px auto auto auto;'>Table 2: Estimated maximum likelihood responses.</p>
</div>

The algorithm does well here and shows how weighting contributions, using inter-worker agreement, can be valuable for judgment aggregation. All the responses derived from the crowd are correct according to our gold labels. For the first four questions, the model has high confidence in its estimate. For question 5, though the majority voted T, the model correctly infers F as the label because worker 1's judgments are given more weight than the others' judgments.

The individual workers' performance estimates, [table 3](#tab3), look good too. Despite the small amount of data, the algorithm successfully figured out that worker 1 was very reliable, workers 2 and 4 had low precision and worker 3 had low recall.

<div id='tab3'>
    <table>
      <tr>
        <th></th>
        <th>Estimated precision</th>
        <th style='background-color: #d6e5d6;'>True precision</th>
        <th>Estimated recall</th>
        <th style='background-color: #d6e5d6;'>True recall</th>
      </tr>
      <tr>
        <td><b>Worker 1</b></td>
        <td style='text-align: center;'>0.99</td>
        <td style='text-align: center; background-color: #ecf5ec;'>1.0</td>
        <td style='text-align: center;'>0.84</td>
        <td style='text-align: center; background-color: #ecf5ec;'>1.0</td>
      </tr>
      <tr>
        <td><b>Worker 2</b></td>
        <td style='text-align: center;'>0.75</td>
        <td style='text-align: center; background-color: #ecf5ec;'>0.67</td>
        <td style='text-align: center;'>0.99</td>
        <td style='text-align: center; background-color: #ecf5ec;'>1.0</td>
      </tr>
      <tr>
        <td><b>Worker 3</b></td>
        <td style='text-align: center;'>0.99</td>
        <td style='text-align: center; background-color: #ecf5ec;'>1.0</td>
        <td style='text-align: center;'>0.48</td>
        <td style='text-align: center; background-color: #ecf5ec;'>0.5</td>
      </tr>
      <tr>
        <td><b>Worker 4</b></td>
        <td style='text-align: center;'>0.12</td>
        <td style='text-align: center; background-color: #ecf5ec;'>0.0</td>
        <td style='text-align: center;'>0.99</td>
        <td style='text-align: center; background-color: #ecf5ec;'>/</td>
      </tr>
    </table>

<p style='text-align: center; font-style: italic; font-size: 13px; max-width: 400px; margin: 10px auto auto auto;'>Table 3: Workers' performance estimates.</p>
</div>

Several extensions of this and other methods [<a href='#ref_raykar'>7</a>, <a href='#ref_whitehill'>8</a>, <a href='#ref_karger'>9</a>, <a href='#ref_zhang'>10</a>] have been proposed for this problem. Zhang et al. [<a href='#ref_zhang'>10</a>], for example, address the issue of getting trapped in a local optimum with EM by computing initial parameter estimates with a spectral method, rather than initializing them randomly. Other approaches focus on modeling micro-task difficulty along with worker reliability [<a href='#ref_whitehill'>8</a>].

## The wisdom of our crowd

There is evidence that, in many settings, a crowd of non-experts can collectively offer estimates that match or exceed those of individual experts [<a href='#ref_surowiecki'>11</a>, <a href='#ref_antweiler'>12</a>, <a href='#ref_chen'>13</a>, <a href='#ref_nofer'>14</a>]. Is this true of our crowd of Figure Eight workers?

To address this question, we applied the EM algorithm as described above. Since many of our disease spans are multi-word expressions, we do this at the token level: for each token selected by at least one worker, its probability to be a disease mention is estimated. 

To begin, we hope that most tokens end up with a probability close to 0 or 1, that is, a high confidence of being part of an entity of interest or not. [Figure 8](#fig8) shows that this is the case.

<div id='fig8'>
    <img src="figures/convergence.png" style='width: 450px; margin-top: 0px;'/>
    <p style='text-align: center; font-style: italic; font-size: 13px; max-width: 400px; margin: 5px auto auto auto;'>Figure 8: The distribution of probabilities estimated with EM, after 1, 2 and 10 iterations (convergence).</p>
</div>

Next, we randomly selected 1,000 tokens from our task and presented them (in their linguistic context) to an in-house expert for labeling. We then compared these expert-provided labels with the ones inferred from the crowd work. [Figure 9](#fig9) summarizes this evaluation, which uses the standard classification metrics of [precision, recall, and F1](https://en.wikipedia.org/wiki/Precision_and_recall). 

<div id='fig9' style='text-align: center;'>
    <img src="figures/perf_thresholds.png" style='width: 600px; margin-top: 0px;'/>
    <p style='text-align: center; font-style: italic; font-size: 13px; margin: 5px auto auto auto;'>Figure 9: Precision, recall and F1 scores at various confidence thresholds for each aggregation method.</p>
</div>

We see that the crowd does well. At a threshold of 0.5, precision and recall are above 0.8, which is comparable to results claimed by MacLean et al. in a similar entity extraction task [<a href='#ref_maclean'>1</a>].  

In this experiment, it appears that most workers have high precision, but recall is very diverse (see [our last section](#Facing-the-crowd)). Therefore, weighting contributions with EM does not really impact the precision of the crowd; majority voting (orange curve) and EM (blue curve) give similar results. For recall, however, since there is high variance among contributors, giving more weight to high-recall workers significantly improves the crowd response.

Assuming a majority of workers are mostly "noise", one could also consider, for each sentence, only the judgment from the most reliable worker according to EM (green curve). In this experiment, this still performs better than majority voting but does not beat the weighted aggregation of all available judgments (blue curve). 

## Facing the crowd

EM estimates individual worker reliability and can therefore help us understand individual behaviors in the crowd. To that end, [figure 10](#fig10) shows the timeline of each work session (a continuous period of work for one contributor).

<div id='fig10'>
    <img src="figures/clusters.png" style='width: 580px; margin-top: 0px;'/>
    <p style='text-align: center; font-style: italic; font-size: 13px; max-width: 400px; margin: 5px auto auto auto;'>Figure 10: Precision and recall against time during work sessions. Clusters are represented with distinct colors.</p>
</div>

As we noted above, most workers have high precision, while recall is very diverse. That is, quality issues mostly come from missing relevant mentions rather than selecting some wrong ones. This is evident in the figure: most of the timelines are near the top of the plot, but they are spread out over the recall axis.

One could think that aggregation could easily fix this bad recall: by combining multiple high-precision, low-recall judgments, we would end up with high-precision, high-recall annotations. We see however that there is a high correlation between workers' behavior; they tend to miss the same text segments. In our task, this has a likely explanation: to maximize profit, workers might choose to increase their judgment rate at the cost of reducing work quality. By reading sentences superficially, they would identify the most obvious relevant mentions while missing more demanding ones. This strategy could go unnoticed during judgment collection because our gold-label assessment deliberately avoids difficult cases and ambiguity.

Using dynamic time warping (DTW) to measure similarity between these time series (see [this blog post for discussion of DTW](https://roamanalytics.com/2016/11/28/shape-matching-with-time-series-data/)), we identified 4 distinct clusters of work sessions. These correspond to the different colors in [figure 10](#fig10):

* Light blue (1,050 timelines): The largest cluster, consisting mostly of short sessions. 

* Yellow (6 timelines): Very low precision and very low recall. These workers unfortunately slipped through our gold-label assessment. 

* Orange (15 timelines): High precision and very low recall, probably from workers who selected almost no text segments. 

* Dark blue (31 timelines): Long sessions consisting of reliable annotations.

This analysis highlights the diversity of the crowd and strengthens our intuition that weighting contributions with individual reliability, estimated using inter-worker agreement, is valuable. A judgment from a dark blue session in [figure 10](#fig10) should be given way more credit than one from the orange cluster.

Although, these findings also reveal that a large group of workers with similar biases might lead to misleading results with EM. For example, a large group of consistently wrong workers would appear to be reliable and might even drown out the work of a smaller, more reliable group. Any unsupervised method will have weaknesses like this, which highlights the importance of having at least some expert annotators reviewing the work.

## Looking ahead

The above findings are reassuring – we might in fact be able to relieve some of the annotation burden from trained experts by using crowdsourcing. It remains to be seen where the boundary lies between work the crowd can do and work that requires experts, especially as we find ways to break down very challenging tasks into simpler component parts that crowdworkers can succeed at.

## Appendix: A closer look to Expectation-Maximization<span id='appendix'></span>

Let's have a closer look to the EM algorithm. We consider an experiment where we want to find the response $r_i$ to questions $i$, using some noisy judgments $J_i$ collected from workers $k$. For each question, we collect boolean judgments $j_{ik}$ (T/F, or None if the worker did not respond to question $i$). Workers reliability is modeled with $\Theta = \theta_1, \ldots, \theta_k$.

We assume judgments are independent between questions for a given worker (no gain of experience) and between workers for a given question (no inter-worker communication).

We describe below the main steps of the algorithm (see [figure 7](#fig7) above).

### Initialization
Since we have no prior knowledge of workers' trustworthiness, their reliability parameters $\Theta$ are initialized to reasonable values based on what we might expect from workers in pools like ours.

### Expectation
During the *expectation* step, workers' performance is re-estimated, given the output from the *maximization* step. Here, we will model workers quality with the true positive, false positive, true negative and false negative rates.

### Maximization
In this step, we calculate, for each question $i$, the maximum likelihood estimate that the true response $r_i$ is positive, given the collected judgments $J_i$: $p(r_i = \text{T} \mid J_i, \Theta)$.  

Using Bayes' theorem: 

$$
\begin{align}
    p(r_i = \text{T} \mid J_i, \Theta) &= \frac{p(J_i \mid r_i = \text{T}, \Theta) p(r_i = \text{T})}{p(J_i \mid \Theta)}
\end{align}
$$

We know that

$$
\begin{align}
    p(J_i \mid \Theta) = 
    p(J_i \mid r_i=\text{T}, \Theta)p(r_i=\text{T}) + 
    p(J_i \mid r_i=\text{F}, \Theta)p(r_i=\text{F})
\end{align}
$$

By substitution, assuming the T and F response outcomes are equiprobable, i.e. $p(r_i = \text{T}) = p(r_i = \text{F})$, we obtain

$$
\begin{align}
p(r_i= \text{T} \mid J_i, \Theta) &= 
\frac{
  p(J_i \mid, r_i=\text{T}, \Theta)
}{
 p(J_i \mid r_i = \text{T}, \Theta) + 
 p(J_i \mid r_i = \text{F}, \Theta)
} \\
\end{align}
$$

Because we assume worker independence, the collected judgment probability for a given question equals the product of individual judgment probabilities:

$$
p(J_i \mid r_i, \Theta) = \prod_{k} p(j_{ik} \mid r_i, \theta_k)
$$

All in all,

$$
\begin{align}
    p(r_i = \text{T} \mid J_i, \Theta) = 
    \prod_{k} 
    \frac{
      p(j_k \mid r_i = \text{T}, \theta_k)
    }{
      p(j_k \mid r_i = \text{T}, \theta_k) + 
      p(j_k \mid r_i = \text{F}, \theta_k)
    }
\end{align}
$$

This probability could be computed from worker's reliability parameters $\Theta$ (estimated in the expectation step). Indeed, $p(j_k \mid r_i = \text{T}, \theta_k)$ is the true positive rate if $j_k = \text{T}$ and the false negative rate if $j_k = \text{F}$. Similarly, for $p(j_k \mid r_i = \text{F}, \theta_k)$, we use the false positive rate if $j_k = \text{T}$ and the true negative rate if $j_k = \text{F}$.

Since it maximizes a non-convex log-likelihood function, this approach has no theoretical guarantee of performance, though empirical studies show that it usually converges to good estimates.

## References

<span id='ref_maclean'>[1] D. MacLean, J. Heer. 2013. Identifying medical terms in patient-authored text: A crowdsourcing-based approach. *Journal of the American Medical Informatics Association (JAMIA)*.</span>

<span id='ref_tenuto'>[2] J. Tenuto. 2015. [How scientists are using CrowdFlower to create a massive biomedical database](https://www.figure-eight.com/citizen-science/). Figure Eight's Artificial Intelligence Resource Center.</span>

<span id='ref_zhai'>[3] H. Zhai, T. Lingren, L. Deleger, Q. Li, M. Kaiser, L Stoutenborough, I Solti. 2013. Web 2.0-based crowdsourcing for high-quality gold standard development in clinical natural language processing. *Journal Medical Internet Research*.

<span id='ref_amerYahia'>[4] S. Amer-Yahia, S. Basu Roy. 2016. Toward worker-centric crowdsourcing. *IEEE Data Engineering Bulletin*.</span>

<span id='ref_aroyo'>[5] L. Aroyo, C. Welty. 2013. Measuring crowd truth for medical relation extraction. In AAAI Fall Symposium on Semantics for Big Data.</span>

<span id='ref_dawidSkene'>[6] A. P. Dawid, A. M. Skene. 1979. Maximum likelihood estimation of observer error-rates using the EM algorithm. Journal of the Royal Statistical Society, Series C.</span>

<span id='ref_raykar'>[7] V. Raykar et al. 2009. Supervised learning from multiple experts: Whom to trust when everyone lies a bit. *International Conference on Machine Learning (ICML)*.</span>

<span id='ref_whitehill'>[8] J. Whitehill, T. Wu, J. Bergsma, J.R. Movellan, P.L. Ruvolo. 2009. Whose vote should count more: Optimal integration of labels from labelers of unknown expertise. *Neural Information Processing Systems (NIPS)*.</span>

<span id='ref_karger'>[9] D.R. Karger, S. Oh, D. Shah. 2011. Iterative learning for reliable crowdsourcing systems. *Neural Information Processing Systems (NIPS)*</span>

<span id='ref_zhang'>[10] Y. Zhang, X. Chen, D. Zhou, M. I. Jordan. 2014. Spectral methods meet EM: A provably optimal algorithm for crowdsourcing. *Neural Information Processing Systems (NIPS)*

<span id='ref_surowiecki'>[11] J. Surowiecki. 2005. *The Wisdom of Crowds*. Anchor Books.</span>

<span id='ref_antweiler'>[12] W. Antweiler, M. Z. Frank. 2004. Is all that talk just noise? The information content of Internet stock message boards. *The Journal of Finance*.</span>

<span id='ref_chen'>[13] H. Chen, P. De, Y. Hu, B.-H. Hwang. 2014. Wisdom of crowds: The value of stock opinions transmitted through social media. *Review of Financial Studies*.</span>

<span id='ref_nofer'>[14] M. Nofer, O. Hinz. 2014. Are crowds on the Internet wiser than experts? The case of a stock prediction community. *Journal of Business Economics*.</span>